In [ ]:
print("Hello, world!")

In [2]:
import os
DEVELOPMENT_MODE = False
# Detect if we're running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
except:
    IN_COLAB = False

# Install if in Colab
if IN_COLAB:
    %pip install transformer_lens
    %pip install circuitsvis
    %pip install pandas
    # Install a faster Node version
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs  # noqa

# Hot reload in development mode & not running on the CD
if not IN_COLAB:
    from IPython import get_ipython
    ip = get_ipython()
    if not ip.extension_manager.loaded:
        ip.extension_manager.load('autoreload')
        %autoreload 2
        
IN_GITHUB = os.getenv("GITHUB_ACTIONS") == "true"




                              DEPRECATION WARNING                            

     Node.js 16.x is no longer actively supported!

  You will not receive security or critical stability updates for this version.

  You should migrate to a supported version of Node.js as soon as possible.
  Use the installation script that corresponds to the version of Node.js you
  wish to install. e.g.
  
   * https://deb.nodesource.com/setup_16.x — Node.js 16 "Gallium" (deprecated)
   * https://deb.nodesource.com/setup_18.x — Node.js 18 "Hydrogen" (Maintenance)
   * https://deb.nodesource.com/setup_19.x — Node.js 19 "Nineteen" (deprecated)
   * https://deb.nodesource.com/setup_20.x — Node.js 20 LTS "Iron" (recommended)
   * https://deb.nodesource.com/setup_21.x — Node.js 21 "Iron" (current)
   


  Please see https://github.com/nodejs/Release for details about which
  version may be appropriate for you.

  The NodeSource Node.js distributions repository contains
  information both about supported ve

In [3]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
if IN_COLAB or not DEVELOPMENT_MODE:
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "notebook_connected"
print(f"Using renderer: {pio.renderers.default}")

Using renderer: colab


In [4]:
# Import stuff
import torch
import torch.nn as nn
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
import plotly.express as px

from jaxtyping import Float
from functools import partial

In [5]:
# import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, FactoredMatrix
import transformer_lens.utils as utils
device = utils.get_device()

In [6]:
torch.set_grad_enabled(False)

torch.autograd.grad_mode.set_grad_enabled(mode=False)

In [7]:
import circuitsvis as cv
# Testing that the library works
cv.examples.hello("George")

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 
target_model_name = "EleutherAI/pythia-410m"
control_model_name = "EleutherAI/pythia-410m-deduped"

target_hf_model = AutoModelForCausalLM.from_pretrained(target_model_name, cache_dir='models/test')
target_tokenizer = AutoTokenizer.from_pretrained(target_model_name)

control_hf_model = AutoModelForCausalLM.from_pretrained(control_model_name, cache_dir='models/test')
control_tokenizer = AutoTokenizer.from_pretrained(control_model_name)

# TRAIN MODEL
# ..continuous pretraining...
#
#
# finised with continuous pretraining

# Save models and tokenizers

target_hf_model.save_pretrained("models/target_hf_model")
target_tokenizer.save_pretrained("models/target_hf_tokenizer")

control_hf_model.save_pretrained("models/control_hf_model")
control_tokenizer.save_pretrained("models/control_hf_tokenizer")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning:


Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).



config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/911M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/911M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

('models/control_hf_tokenizer/tokenizer_config.json',
 'models/control_hf_tokenizer/special_tokens_map.json',
 'models/control_hf_tokenizer/tokenizer.json')

In [9]:
# LOAD TRAINED MODELS AND USE WITH TRANSFORMERLENS
from transformers import AutoModelForCausalLM, AutoTokenizer

loaded_target_hf_model = AutoModelForCausalLM.from_pretrained("models/target_hf_model")
loaded_target_tokenizer = AutoTokenizer.from_pretrained("models/target_hf_tokenizer")

loaded_control_hf_model = AutoModelForCausalLM.from_pretrained("models/control_hf_model")
loaded_control_tokenizer = AutoTokenizer.from_pretrained("models/control_hf_tokenizer")


In [10]:
from transformer_lens import HookedTransformer

# load into TransformerLens
tl_control_model = HookedTransformer.from_pretrained(
    control_model_name,
    hf_model=loaded_control_hf_model,
    tokenizer=loaded_control_tokenizer,
    device=device
)

tl_target_model = HookedTransformer.from_pretrained(
    target_model_name,
    hf_model=loaded_target_hf_model,
    tokenizer=loaded_target_tokenizer,
    device=device
)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Loaded pretrained model EleutherAI/pythia-410m-deduped into HookedTransformer


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Loaded pretrained model EleutherAI/pythia-410m into HookedTransformer


In [12]:
test_string = "John Doe worked at the supermarket as a butcher. At his work he helps clients select the best cuts of"
tokens = tl_control_model.to_tokens(test_string)
print(f"model tokens: {tokens}")
print(f"tokenized text: {tl_control_model.to_str_tokens(test_string)}")

model tokens: tensor([[    0,  8732, 34365,  4307,   387,   253, 39556,   347,   247, 45975,
            15,  2058,   521,   789,   344,  7729,  8548,  3609,   253,  1682,
         12176,   273]], device='cuda:0')
tokenized text: ['<|endoftext|>', 'John', ' Doe', ' worked', ' at', ' the', ' supermarket', ' as', ' a', ' butcher', '.', ' At', ' his', ' work', ' he', ' helps', ' clients', ' select', ' the', ' best', ' cuts', ' of']


In [13]:
target_logits, target_cache = tl_target_model.run_with_cache(tokens)

# Get the logits for the last token position
last_token_logits = target_logits[0, -1, :]  # Shape: [vocab_size]

# Get the correct token (highest probability token)
correct_token_id = torch.argmax(last_token_logits).item()
correct_token = tl_target_model.tokenizer.decode([correct_token_id])

print(f"Target model's predicted token: {correct_token} (ID: {correct_token_id})")
print(f"Target model's probability for this token: {torch.softmax(last_token_logits, dim=-1)[correct_token_id].item():.4f}")


Target model's predicted token:  meat (ID: 9132)
Target model's probability for this token: 0.7639


In [15]:
# see what layers are available to patch
layer_names = [name for name in target_cache.keys()]
# Group layer names by block (e.g., layer0, layer1, ...)
from collections import defaultdict
import re

blocks = defaultdict(list)
block_pattern = re.compile(r'(layer\d+|blocks\.\d+)')

for name in layer_names:
    match = block_pattern.search(name)
    if match:
        block = match.group(0)
    else:
        block = 'other'
    blocks[block].append(name)

for block in sorted(blocks.keys()):
    print(f"\nBlock: {block}")
    for lname in blocks[block]:
        print(f"  {lname}")


Block: blocks.0
  blocks.0.hook_resid_pre
  blocks.0.ln1.hook_scale
  blocks.0.ln1.hook_normalized
  blocks.0.attn.hook_q
  blocks.0.attn.hook_k
  blocks.0.attn.hook_v
  blocks.0.attn.hook_rot_q
  blocks.0.attn.hook_rot_k
  blocks.0.attn.hook_attn_scores
  blocks.0.attn.hook_pattern
  blocks.0.attn.hook_z
  blocks.0.hook_attn_out
  blocks.0.ln2.hook_scale
  blocks.0.ln2.hook_normalized
  blocks.0.mlp.hook_pre
  blocks.0.mlp.hook_post
  blocks.0.hook_mlp_out
  blocks.0.hook_resid_post

Block: blocks.1
  blocks.1.hook_resid_pre
  blocks.1.ln1.hook_scale
  blocks.1.ln1.hook_normalized
  blocks.1.attn.hook_q
  blocks.1.attn.hook_k
  blocks.1.attn.hook_v
  blocks.1.attn.hook_rot_q
  blocks.1.attn.hook_rot_k
  blocks.1.attn.hook_attn_scores
  blocks.1.attn.hook_pattern
  blocks.1.attn.hook_z
  blocks.1.hook_attn_out
  blocks.1.ln2.hook_scale
  blocks.1.ln2.hook_normalized
  blocks.1.mlp.hook_pre
  blocks.1.mlp.hook_post
  blocks.1.hook_mlp_out
  blocks.1.hook_resid_post

Block: blocks.10
  

In [20]:
# Define activation patching function
def patch_activation_at_layer(
    activation,  # The activation to patch in
    hook,  # Hook point object
    target_cache,  # Cache from target model
    patch_layer  # Which layer to patch
):
    """
    Patches activations at a specific layer by replacing them with target model activations.
    """
    # Check if this is the layer we want to patch
    if hook.name == patch_layer:
        # Replace with target model's activations at this layer
        patched_activation = target_cache[patch_layer]
        # Verify shapes match (they should if models have same architecture)
        if patched_activation.shape != activation.shape:
            raise ValueError(f"Shape mismatch at {patch_layer}: target {patched_activation.shape} vs control {activation.shape}")
        return patched_activation
    return activation

# Get all layer names that we can patch
# We'll patch residual stream activations after each layer
layer_names = [name for name in target_cache.keys() if 'resid_post' in name ]
# Also include attention and MLP intermediate activations
# layer_names += [name for name in target_cache.keys() if 'hook' in name and ('attn' in name or 'mlp' in name)]

# Sort to get a consistent order
layer_names = sorted(set(layer_names))
print(f"Found {len(layer_names)} hook points to patch")
print(f"Sample hook points: {layer_names[:5]}")


Found 24 hook points to patch
Sample hook points: ['blocks.0.hook_resid_post', 'blocks.1.hook_resid_post', 'blocks.10.hook_resid_post', 'blocks.11.hook_resid_post', 'blocks.12.hook_resid_post']


In [28]:
# Perform activation patching for each layer
results = []

# First, get baseline (control model without patching)
control_logits_baseline, _ = tl_control_model.run_with_cache(tokens)
control_probs_baseline = torch.softmax(control_logits_baseline[0, -1, :], dim=-1)
baseline_prob = control_probs_baseline[correct_token_id].item()
results.append({
    'layer': 'baseline (no patch)',
    'prob_correct_token': baseline_prob,
    'prob_change': 0.0
})

print(f"Baseline (control model, no patching): P(correct_token) = {baseline_prob:.4f}")

# Now patch each layer
for layer_name in tqdm.tqdm(layer_names):
    # Create a hook function that patches this specific layer
    # Use a closure to capture the layer_name correctly
    def make_patch_fn(layer):
        def patch_fn(activation, hook):
            return patch_activation_at_layer(activation, hook, target_cache, layer)
        return patch_fn
    
    # Run control model with patching at this layer
    # Only patch if the layer exists in both caches
    if layer_name not in target_cache:
        print(f"Warning: {layer_name} not found in target cache, skipping")
        continue
        
    control_logits_patched = tl_control_model.run_with_hooks(
        tokens,
        fwd_hooks=[(layer_name, make_patch_fn(layer_name))],
        return_type='logits'
    )
    print(f"control_logits_patched.shape: {control_logits_patched.shape}")
    
    # Get probability of correct token
    control_probs_patched = torch.softmax(control_logits_patched[0, -1, :], dim=-1)
    prob_correct = control_probs_patched[correct_token_id].item()
    prob_change = prob_correct - baseline_prob
    
    results.append({
        'layer': layer_name,
        'prob_correct_token': prob_correct,
        'prob_change': prob_change
    })

print(f"\nCompleted activation patching for {len(layer_names)} layers")
print(results)



Baseline (control model, no patching): P(correct_token) = 0.8779


  0%|          | 0/24 [00:00<?, ?it/s]

control_logits_patched.shape: torch.Size([1, 22, 50304])
control_logits_patched.shape: torch.Size([1, 22, 50304])
control_logits_patched.shape: torch.Size([1, 22, 50304])
control_logits_patched.shape: torch.Size([1, 22, 50304])
control_logits_patched.shape: torch.Size([1, 22, 50304])
control_logits_patched.shape: torch.Size([1, 22, 50304])
control_logits_patched.shape: torch.Size([1, 22, 50304])
control_logits_patched.shape: torch.Size([1, 22, 50304])
control_logits_patched.shape: torch.Size([1, 22, 50304])
control_logits_patched.shape: torch.Size([1, 22, 50304])
control_logits_patched.shape: torch.Size([1, 22, 50304])
control_logits_patched.shape: torch.Size([1, 22, 50304])
control_logits_patched.shape: torch.Size([1, 22, 50304])
control_logits_patched.shape: torch.Size([1, 22, 50304])
control_logits_patched.shape: torch.Size([1, 22, 50304])
control_logits_patched.shape: torch.Size([1, 22, 50304])
control_logits_patched.shape: torch.Size([1, 22, 50304])
control_logits_patched.shape: t

In [ ]:
# Visualize results
import pandas as pd

df_results = pd.DataFrame(results)

# Sort by probability change (most positive first)
df_results_sorted = df_results.sort_values('prob_change', ascending=False)

print("Top 10 layers by probability increase:")
print(df_results_sorted.head(10)[['layer', 'prob_correct_token', 'prob_change']].to_string(index=False))

print("\n" + "="*80)
print("Bottom 10 layers by probability change:")
print(df_results_sorted.tail(10)[['layer', 'prob_correct_token', 'prob_change']].to_string(index=False))



In [ ]:
# Plot results
fig = px.bar(
    df_results_sorted[df_results_sorted['layer'] != 'baseline (no patch)'],
    x='layer',
    y='prob_change',
    title=f'Activation Patching: Change in P(correct_token="{correct_token}")',
    labels={'prob_change': 'Change in Probability', 'layer': 'Layer Name'},
    color='prob_change',
    color_continuous_scale='RdYlGn'
)
fig.update_xaxes(tickangle=45)
fig.update_layout(height=600, showlegend=False)
fig.show()

# Also plot absolute probabilities
fig2 = px.bar(
    df_results_sorted,
    x='layer',
    y='prob_correct_token',
    title=f'Activation Patching: P(correct_token="{correct_token}")',
    labels={'prob_correct_token': 'Probability', 'layer': 'Layer Name'},
    color='prob_correct_token',
    color_continuous_scale='Viridis'
)
fig2.update_xaxes(tickangle=45)
fig2.update_layout(height=600, showlegend=False)
fig2.show()



In [ ]:
# Summary of key findings
print("="*80)
print("ACTIVATION PATCHING SUMMARY")
print("="*80)
print(f"\nTarget model predicted token: '{correct_token}' (ID: {correct_token_id})")
print(f"Target model probability: {torch.softmax(target_logits[0, -1, :], dim=-1)[correct_token_id].item():.4f}")
print(f"Control model baseline probability: {baseline_prob:.4f}")

# Find layers with biggest impact
df_results_sorted = df_results.sort_values('prob_change', ascending=False)
top_positive = df_results_sorted[df_results_sorted['prob_change'] > 0].head(5)
top_negative = df_results_sorted[df_results_sorted['prob_change'] < 0].tail(5)

if len(top_positive) > 0:
    print(f"\nTop 5 layers that INCREASE probability:")
    for idx, row in top_positive.iterrows():
        print(f"  {row['layer']}: +{row['prob_change']:.4f} (final prob: {row['prob_correct_token']:.4f})")

if len(top_negative) > 0:
    print(f"\nTop 5 layers that DECREASE probability:")
    for idx, row in top_negative.iterrows():
        print(f"  {row['layer']}: {row['prob_change']:.4f} (final prob: {row['prob_correct_token']:.4f})")

print("\n" + "="*80)



In [ ]:
# CircuitsVis visualizations for activation patching
import circuitsvis as cv
import numpy as np

# Get token strings for visualization
if tokens is not None:
    token_strs = tl_control_model.to_str_tokens(test_string)
else:
    token_strs = tl_control_model.to_str_tokens(test_string)

print(f"Tokens: {token_strs}")
print(f"Number of tokens: {len(token_strs)}")



In [ ]:
# Create a heatmap showing activation patching effects across layers
# Prepare data: layer names vs probability changes
layer_names_only = [r['layer'] for r in results if r['layer'] != 'baseline (no patch)']
prob_changes = [r['prob_change'] for r in results if r['layer'] != 'baseline (no patch)']

# Create a matrix for visualization (we can show this as a bar chart or heatmap)
# For circuitsvis, we'll create token-level visualizations showing the impact

# First, let's visualize the top layers that affect the prediction
top_layers = df_results_sorted[df_results_sorted['layer'] != 'baseline (no patch)'].head(10)

print("Visualizing top 10 layers with circuitsvis:")
for idx, row in top_layers.iterrows():
    layer_name = row['layer']
    prob_change = row['prob_change']
    print(f"  {layer_name}: {prob_change:.4f}")



In [ ]:
# Visualize activation differences using circuitsvis
# We'll create visualizations for the most impactful layers

# Get attention patterns for comparison (if available)
target_attention = None
control_attention = None

# Try to extract attention patterns from existing caches
try:
    # Attention is typically stored in cache with keys like "blocks.X.attn.hook_pattern"
    # or "blocks.X.attn.hook_attn" in TransformerLens
    attn_keys = [k for k in target_cache.keys() if 'attn.hook_pattern' in k or 'attn.hook_attn' in k or 'pattern' in k]
    if attn_keys:
        # Get attention from the last layer as an example
        last_attn_key = sorted(attn_keys)[-1]
        target_attention_raw = target_cache[last_attn_key]
        # Handle different tensor shapes: [batch, head, seq, seq] or [head, seq, seq]
        if len(target_attention_raw.shape) == 4:
            target_attention = target_attention_raw[0].detach().cpu().numpy()  # [head, seq, seq]
        else:
            target_attention = target_attention_raw.detach().cpu().numpy()
        print(f"Found target attention pattern: {last_attn_key}, shape: {target_attention.shape}")
        
        # Get control model attention from baseline cache
        control_logits_baseline, control_cache_baseline = tl_control_model.run_with_cache(control_tokens)
        control_attn_keys = [k for k in control_cache_baseline.keys() if 'attn.hook_pattern' in k or 'attn.hook_attn' in k or 'pattern' in k]
        if control_attn_keys:
            control_last_attn_key = sorted(control_attn_keys)[-1]
            control_attention_raw = control_cache_baseline[control_last_attn_key]
            if len(control_attention_raw.shape) == 4:
                control_attention = control_attention_raw[0].detach().cpu().numpy()  # [head, seq, seq]
            else:
                control_attention = control_attention_raw.detach().cpu().numpy()
            print(f"Found control attention pattern: {control_last_attn_key}, shape: {control_attention.shape}")
    else:
        print("No attention patterns found in cache. Available keys:", list(target_cache.keys())[:10])
except Exception as e:
    print(f"Could not extract attention patterns: {e}")
    import traceback
    traceback.print_exc()



In [ ]:
# Visualize attention patterns with circuitsvis (if available)
if target_attention is not None and control_attention is not None:
    print("\n" + "="*80)
    print("ATTENTION PATTERN VISUALIZATION")
    print("="*80)
    
    # Average across heads for visualization
    target_attn_avg = target_attention.mean(axis=0)  # [seq, seq]
    control_attn_avg = control_attention.mean(axis=0)  # [seq, seq]
    
    # Visualize target model attention
    print("\nTarget Model Attention Pattern (last layer, averaged across heads):")
    cv.attention.attention_heads(
        attention=target_attention,  # [head, seq, seq]
        tokens=token_strs,
    )
    
    print("\nControl Model Attention Pattern (last layer, averaged across heads):")
    cv.attention.attention_heads(
        attention=control_attention,  # [head, seq, seq]
        tokens=token_strs,
    )
    
    # Show attention difference
    attn_diff = target_attn_avg - control_attn_avg
    print("\nAttention Difference (Target - Control):")
    # For circuitsvis, we need to provide it in the right format
    # Create a single "head" with the difference
    attn_diff_heads = attn_diff[np.newaxis, :, :]  # [1, seq, seq]
    cv.attention.attention_heads(
        attention=attn_diff_heads,
        tokens=token_strs,
    )
else:
    print("Attention patterns not available for visualization")



In [ ]:
# Create a token-level visualization showing which tokens are most important
# We'll visualize the logit differences for the correct token

# Get logits for all tokens from baseline and best patched layer
best_layer_row = df_results_sorted[df_results_sorted['layer'] != 'baseline (no patch)'].iloc[0]
best_layer_name = best_layer_row['layer']

print(f"\nVisualizing impact of patching layer: {best_layer_name}")
print(f"Probability change: {best_layer_row['prob_change']:.4f}")

# Re-run to get full logit distribution for the best layer
def make_patch_fn_best(layer):
    def patch_fn(activation, hook):
        return patch_activation_at_layer(activation, hook, target_cache, layer)
    return patch_fn

best_logits, _ = tl_control_model.run_with_cache(
    control_tokens,
    fwd_hooks=[(best_layer_name, make_patch_fn_best(best_layer_name))]
)

baseline_logits, _ = tl_control_model.run_with_cache(control_tokens)

# Get logit differences for the correct token at each position
# (though we mainly care about the last position)
last_pos_baseline_logits = baseline_logits[0, -1, :].detach().cpu().numpy()
last_pos_best_logits = best_logits[0, -1, :].detach().cpu().numpy()

# Get top tokens that changed the most
top_k = 20
top_tokens_baseline = np.argsort(last_pos_baseline_logits)[-top_k:][::-1]
top_tokens_best = np.argsort(last_pos_best_logits)[-top_k:][::-1]

# Show token probabilities
print(f"\nTop {top_k} tokens (baseline control model):")
baseline_probs = torch.softmax(baseline_logits[0, -1, :], dim=-1).detach().cpu().numpy()
for token_id in top_tokens_baseline[:10]:
    token_str = tl_control_model.tokenizer.decode([token_id])
    prob = baseline_probs[token_id]
    is_correct = "✓" if token_id == correct_token_id else " "
    print(f"  {is_correct} {token_str:20s} (ID: {token_id:5d}): {prob:.4f}")

print(f"\nTop {top_k} tokens (with patching at {best_layer_name}):")
best_probs = torch.softmax(best_logits[0, -1, :], dim=-1).detach().cpu().numpy()
for token_id in top_tokens_best[:10]:
    token_str = tl_control_model.tokenizer.decode([token_id])
    prob = best_probs[token_id]
    is_correct = "✓" if token_id == correct_token_id else " "
    print(f"  {is_correct} {token_str:20s} (ID: {token_id:5d}): {prob:.4f}")



In [ ]:
# Create a colored tokens visualization showing the input tokens
# We can color them based on how much they contribute to the final prediction

# For each input token position, we can compute how much patching affects the output
# This is a simplified version - in practice, you might want to patch at each position separately

# Create a simple visualization: color tokens by their position
# (in a more sophisticated version, you could compute position-specific effects)

# Get the token strings
input_tokens = token_strs

# Create values for coloring (for now, just show all tokens equally)
# In a more advanced version, you could compute per-token attribution
token_values = [1.0] * len(input_tokens)  # Placeholder

print("\nInput tokens visualization:")
cv.tokens.colored_tokens(
    tokens=input_tokens,
    values=token_values,
    color_map="RdBu"
)

# Also show which token is being predicted
print(f"\nPredicted token: '{correct_token}'")
print(f"Baseline probability: {baseline_prob:.4f}")
print(f"Best patched probability: {best_layer_row['prob_correct_token']:.4f}")



In [ ]:
# Create a heatmap visualization of activation patching effects
# Prepare data for a layer-by-layer heatmap

# Extract layer numbers from layer names for better visualization
def extract_layer_num(layer_name):
    """Extract layer number from layer name like 'blocks.5.attn.hook_q' -> 5"""
    import re
    match = re.search(r'blocks\.(\d+)', layer_name)
    if match:
        return int(match.group(1))
    match = re.search(r'\.(\d+)\.', layer_name)
    if match:
        return int(match.group(1))
    return -1

# Create a matrix: layer number vs probability change
layer_data = []
for r in results:
    if r['layer'] != 'baseline (no patch)':
        layer_num = extract_layer_num(r['layer'])
        if layer_num >= 0:
            layer_data.append({
                'layer_num': layer_num,
                'layer_name': r['layer'],
                'prob_change': r['prob_change'],
                'prob_abs': r['prob_correct_token']
            })

if layer_data:
    layer_df = pd.DataFrame(layer_data)
    layer_df = layer_df.sort_values('layer_num')
    
    print("\nLayer-by-layer activation patching effects:")
    print("="*80)
    
    # Group by layer number and show summary
    for layer_num in sorted(layer_df['layer_num'].unique()):
        layer_rows = layer_df[layer_df['layer_num'] == layer_num]
        max_change_row = layer_rows.loc[layer_rows['prob_change'].idxmax()]
        print(f"Layer {layer_num:2d}: max change = {max_change_row['prob_change']:+.4f} "
              f"({max_change_row['layer_name']})")
    
    # Create a visualization showing layer effects
    # We can use circuitsvis or plotly for this
    print("\nCreating layer effect visualization...")

